Este script permite capturar las notificaciones enviadas por 
una camara Hikvision frente a un evento, tal como deteccion de movimiento

 ## Configurar la camara

Antes es necesario configurar la Camara Hikvision para recibir las notificaciones
debemos tener un archivo XML con la siguiente informacion:

#### server_events.xml 
```xml
<HttpHostNotification version="2.0">
<id>100</id>
<url>eventos</url>
<protocolType>HTTP</protocolType>
<parameterFormatType>XML</parameterFormatType>
<addressingFormatType>ipaddress</addressingFormatType>
<ipAddress>192.168.0.1</ipAddress>
<portNo>8000</portNo>
<userName></userName>
<httpAuthenticationMethod>none</httpAuthenticationMethod>
</HttpHostNotification>"
```
Como parametro importantes son la IP y puerto del servidor, la funciona a ejecutar(eventos)

Esta configuracion se cargara con el siguiente comando curl:

 curl -T server_events.xml http://usuario:password@ip_camara/ISAPI/Event/notification/httpHosts/1
 
Ademas es necesario configurar la deteccion de movimiento


### Servicio de recepcion

El siguiente script captura los eventos de las camaras

In [125]:
# Librerias necesarias 
from flask import Flask
from flask import request
import xmltodict

In [126]:
'''
Formato de notificacion XML de camara hikvision

<?xml version="1.0" encoding="UTF-8"?>
<EventNotificationAlert version="1.0" xmlns="urn:psialliance-org">
<ipAddress>10.0.20.30</ipAddress>
<protocolType>HTTP</protocolType>
<macAddress>a4:14:37:7f:5a:a6</macAddress>
<channelID>1</channelID>
<dateTime>2020-04-24T15:31:38-4:00</dateTime>
<activePostCount>1474</activePostCount>
<eventType>VMD</eventType>
<eventState>active</eventState>
<eventDescription>Motion alarm</eventDescription>
<Extensions version="1.0" xmlns="urn:psialliance-org">
<serialNumber xmlns="urn:selfextension:psiaext-ver10-xsd">DS-2CD2120F-IS20161028AAWR665222292</serialNumber>
<eventPush xmlns="urn:selfextension:psiaext-ver10-xsd">VMD  DS-2CD2120F-IS20161028AAWR665222292,2020-04-24T15:31:38-4:00,1,1.0</eventPush>
</Extensions>
</EventNotificationAlert>
'''

'\nFormato de notificacion XML de camara hikvision\n\n<?xml version="1.0" encoding="UTF-8"?>\n<EventNotificationAlert version="1.0" xmlns="urn:psialliance-org">\n<ipAddress>10.0.20.30</ipAddress>\n<protocolType>HTTP</protocolType>\n<macAddress>a4:14:37:7f:5a:a6</macAddress>\n<channelID>1</channelID>\n<dateTime>2020-04-24T15:31:38-4:00</dateTime>\n<activePostCount>1474</activePostCount>\n<eventType>VMD</eventType>\n<eventState>active</eventState>\n<eventDescription>Motion alarm</eventDescription>\n<Extensions version="1.0" xmlns="urn:psialliance-org">\n<serialNumber xmlns="urn:selfextension:psiaext-ver10-xsd">DS-2CD2120F-IS20161028AAWR665222292</serialNumber>\n<eventPush xmlns="urn:selfextension:psiaext-ver10-xsd">VMD  DS-2CD2120F-IS20161028AAWR665222292,2020-04-24T15:31:38-4:00,1,1.0</eventPush>\n</Extensions>\n</EventNotificationAlert>\n'

In [127]:
app = Flask(__name__)
 
@app.route('/evento', methods = ['POST','GET'])
def evento():
    content = request
    print(request)
    ## se captura los datos en XML y se transporma a diccionario
    datos=request.get_data()
    datos_string = datos.decode('utf-8')
    datos_string = datos_string.replace('&', ' ')
    datos_dic = xmltodict.parse(datos_string)
    ## se imprimen los datos principales
    print("Camara: " + datos_dic["EventNotificationAlert"]["ipAddress"])
    print("Canal: " + datos_dic["EventNotificationAlert"]["channelID"])
    print("Tipo de evento: " + datos_dic["EventNotificationAlert"]["eventType"])
    print("Estado: " + datos_dic["EventNotificationAlert"]["eventState"])
    print("Descripcion: " + datos_dic["EventNotificationAlert"]["eventDescription"])
    return 'JSON posted' 
app.run(host='0.0.0.0', port= 8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
